# Visualization Strawman

(23-May-2022 jat) This is an exemplar for some of the features to be prototyped in Phase 1. The main goal of our software should be to make it **easy** for notebook authors to render 3-D data arrays.

## 1. Load test data

* Set `dataset_path` to the path on your file system
* Future: make datasets like this available on kwnas and/or internal http server?

In [1]:
# Load test dataset
import xarray as xr
import zarr

dataset_path = '/home/john/projects/pan3d/data/test'
ds = xr.open_zarr(dataset_path, consolidated=False)

## 2. Select data array

In [2]:
temp_array = ds.T
temp_array

<xarray.DataArray 'T' (time: 396, Z: 40, XC_agg: 20, YC_agg: 20)>
[6336000 values with dtype=float32]
Coordinates:
    PHrefC   (Z) float32 ...
  * XC_agg   (XC_agg) float32 5e+04 1.5e+05 2.5e+05 ... 1.85e+06 1.95e+06
  * YC_agg   (YC_agg) float32 5e+04 1.5e+05 2.5e+05 ... 1.85e+06 1.95e+06
  * Z        (Z) float32 -5.0 -15.0 -25.0 ... -2.728e+03 -2.83e+03 -2.934e+03
    drF      (Z) float32 ...
    dxCc     (YC_agg, XC_agg) float32 ...
    dyCc     (YC_agg, XC_agg) float32 ...
    hFacC    (Z, XC_agg, YC_agg) float32 ...
  * time     (time) timedelta64[ns] 61023 days 61038 days ... 66948 days

## 3. Create vtkDataObject instance from data array at one timestep
* Presumes there is a hypothetical `pan3d` module that encapsulates vtk and pyvista as needed.
* So no, this code won't run.
* `pan3d.to_vtk()` recognizes that the data array represents a rectilinear grid so, in this case, it returns a `vtkRectiliearGrid` object.
* Technically, the code returns a python subclass of `vtkRectilinearGrid` that uses an xarray accessor (to be developed) so that no data copying is required.

In [3]:
import pan3d_mockup as pan3d

temp_grid = pan3d.to_vtk(temp_array[0]) # returns vtkRectilinearGrid (subclass)
temp_grid

RectilinearGrid (0x7f6d6a6b2340)
  N Cells:	16000
  N Points:	18081
  X Bounds:	1.667e+02, 6.833e+03
  Y Bounds:	4.167e+03, 1.083e+04
  Z Bounds:	-2.985e+03, -5.000e+00
  Dimensions:	21, 21, 41
  N Arrays:	1

In [30]:
# And display, of course
pan3d.plot(temp_grid, clim=[0, 8], cmap='magma', show_edges=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## 4. Plot data array at one timestep
* When an xarray DataArray is passed in, `pan3d.plot()` internally creates the corresponding `vtkDataObject` instance and rendering pipeline.
* `pan3d.plot()` options are TBD but should at least include color map, opacity map, client or server rendering mode, ...

In [4]:
eddy_array = ds.eddyV
i = len(eddy_array.time) // 2  # use middle time index
pan3d.plot(eddy_array[i], cmap='RdBu_r', clim=[-2e5, 2e5], show_edges=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## 5. Plot data array with timestep control
* When the data array includes time coordinates, `pan3d.plot()` displays the first time step (like paraview). Maybe it should also display a default time selector?


In [27]:
plotter = pan3d.Plotter(temp_array, show_edges=True)
plotter.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## 6. Attach a slider to set the timestep
* Our dataset doesn't appear to have meaningful calendar dates, so just using index for this example.
* Call `Plotter.set_index()` to change the timestep
* Future: add `set_date()` method and/or maybe `setTime()`
* Note that you have to look carefully to see the data change when using the slider. Internal code might have bugs.
* Future: pan3d provides a calendar-aware slider that includes labeled tick marks on the axis (days, months, years, depending on range). Should also have popup calendar picker on double-click.

### <span style="color: red;">WARNING: THE `Plotter.set_index()` METHOD LEAKS MEMORY</span>

In [31]:
import ipywidgets as widgets

max_index = ds.time.size - 1
widgets.interact(plotter.set_index, i=widgets.IntSlider(min=0, max=max_index, step=1, value=10, layout=widgets.Layout(width='95%')));

interactive(children=(IntSlider(value=10, description='i', layout=Layout(width='95%'), max=395), Output()), _d…